# How to pass multimodal data directly to models

Here we demonstrate how to pass multimodal input directly to models. 
We currently expect all input to be passed in the same format as [OpenAI expects](https://platform.openai.com/docs/guides/vision).
For other model providers that support multimodal input, we have added logic inside the class to convert to the expected format.

In this example we will ask a model to describe an image.

In [1]:
image_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg"

In [2]:
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o")

The most commonly supported way to pass in images is to pass it in as a byte string.
This should work for most model integrations.

In [3]:
import base64

import httpx

image_data = base64.b64encode(httpx.get(image_url).content).decode("utf-8")

In [4]:
message = HumanMessage(
    content=[
        {"type": "text", "text": "describe the weather in this image"},
        {
            "type": "image_url",
            "image_url": {"url": f"data:image/jpeg;base64,{image_data}"},
        },
    ],
)
response = model.invoke([message])
print(response.content)

The weather in the image appears to be clear and pleasant. The sky is mostly blue with some scattered white clouds, suggesting it is likely a sunny day with minimal cloud cover. There is no indication of precipitation or severe weather. The lighting suggests that the sun is shining, likely contributing to a warm and comfortable temperature. The lush green grass and vibrant colors also hint at favorable weather conditions.


We can feed the image URL directly in a content block of type "image_url". Note that only some model providers support this.

In [5]:
message = HumanMessage(
    content=[
        {"type": "text", "text": "describe the weather in this image"},
        {"type": "image_url", "image_url": {"url": image_url}},
    ],
)
response = model.invoke([message])
print(response.content)

The weather in the image appears to be clear and sunny. The sky is mostly blue with some scattered clouds, suggesting it is a pleasant day with good visibility. There is no indication of rain or stormy conditions, and the green grass and lush vegetation suggest that the weather has been favorable for plant growth. Overall, it looks like a warm and bright day.


We can also pass in multiple images.

In [6]:
message = HumanMessage(
    content=[
        {"type": "text", "text": "are these two images the same?"},
        {"type": "image_url", "image_url": {"url": image_url}},
        {"type": "image_url", "image_url": {"url": image_url}},
    ],
)
response = model.invoke([message])
print(response.content)

Yes, the two images are the same. They have identical scenery with the same path, grass, trees, and sky.


## Tool calls

Some multimodal models support [tool calling](/docs/concepts/#functiontool-calling) features as well. To call tools using such models, simply bind tools to them in the [usual way](/docs/how_to/tool_calling), and invoke the model using content blocks of the desired type (e.g., containing image data).

In [7]:
from typing import Literal

from langchain_core.tools import tool


@tool
def weather_tool(weather: Literal["sunny", "cloudy", "rainy"]) -> None:
    """Describe the weather"""
    pass


model_with_tools = model.bind_tools([weather_tool])

message = HumanMessage(
    content=[
        {"type": "text", "text": "describe the weather in this image"},
        {"type": "image_url", "image_url": {"url": image_url}},
    ],
)
response = model_with_tools.invoke([message])
print(response.tool_calls)

[{'name': 'weather_tool', 'args': {'weather': 'sunny'}, 'id': 'call_lGusTdfjRk2hDtUpQSJxz8of', 'type': 'tool_call'}]
